In [25]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

👆

`/tmp/ipykernel_141638/1646494204.py:2: 用户警告：命名张量及其所有相关的API都是实验性功能，可能会发生变化。在它们被发布为稳定版本之前，请不要用它们来做任何重要的事情。（在../c10/core/TensorImpl.h:1788内部触发） _ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])`

In [26]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.71152, 0.0722])

`img_t = torch.randn(3, 5, 5)` 

shape: [channels, rows, columns]

channels: RGB

---

`weights = torch.tensor([0.2126, 0.7152, 0.0722])`

其中, 0.2126, 0.7152, 0.0722 是 RGB 三个通道的权重

> - 0.2126: 红色
> - 0.7152: 绿色
> - 0.0722: 蓝色

In [27]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

`[多少张图片, 通道数, 行, 列]`

## 把三维图片转换成一维(RGB 👉 灰度)

### 平均

In [28]:
img_gray_naive = img_t.mean(-3)  # 既是把 '通道' 维度的 RGB 三个数求平均
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

👆

对于一个倒数第三个维度取平均之后就变成了一个0维的, 相当于是, 一个一维的向量取平均之后就变成0维的了

---

### 加权平均

观察 `weights = torch.tensor([0.2126, 0.71152, 0.0722])`, 对此, 其为一维的, 这不能直接进行简单的相加, 由此, 需要用到之前的 `unsqueeze` 方法, 将其变为和图片一样的三维的, 再进行相关处理.

In [29]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

👆

#### P.S.

`unsqueeze()` 和 `unsqueeze_()` 区别:

- `unsqueeze()` 返回一个新的张量, 而 `unsqueeze_()` 则是直接在原来的张量上进行操作, 也就是说 `unsqueeze()` 会占用更多的内存空间, 而 `unsqueeze_()` 则不会. 所以 `unsqueeze_()` 再原地操作 (In-place) 的时候, 会更加高效.
- 同样的, 如果想要将修改后的张量赋值给别的变量, 则需要使用 `unsqueeze()`, 而不能使用 `unsqueeze_()`

对此, 我们可以简单的用下面的代码进行测试: 

👇

In [30]:
# unsqueeze()
a = weights.clone()
b = a.unsqueeze(-1)

# unsqueeze_()
c = weights.clone()
d = c.unsqueeze_(-1)
a == b, c == d

(tensor([[ True, False, False],
         [False,  True, False],
         [False, False,  True]]),
 tensor([[True],
         [True],
         [True]]))

In [34]:
img_weights = (img_t * unsqueezed_weights)
batch_weigts = (batch_t * unsqueezed_weights)
img_weights.shape, batch_weigts.shape

(torch.Size([3, 5, 5]), torch.Size([2, 3, 5, 5]))

In [36]:
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weigts.sum(-3)
img_gray_weighted.shape, batch_gray_weighted.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

### 爱因斯坦求和

In [38]:
img_gray_weighted_fancy = torch.einsum('...chw, c-> ...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw, c-> ...hw', batch_t, weights)
img_gray_weighted_fancy.shape, batch_gray_weighted_fancy.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

👆

`...chw`, 是倒着看的, 也就是从后往前数, 意思就是就算 `...` 之前还有一个维度也不需要写出来, 而是依旧从后往前的 `chw`

## 命名

In [39]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0772], names = ['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0772], names=('channels',))

In [41]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')

In [42]:
img_named

tensor([[[ 0.6416,  0.1317, -1.0025, -0.5073,  0.3314],
         [-1.0648,  2.0866,  0.4873,  0.2159,  0.5916],
         [ 0.9874,  0.8971,  0.0078,  0.7424, -1.3605],
         [-0.0361, -0.2031,  1.0998,  0.7136, -0.4406],
         [ 1.0139,  1.4851, -1.2603, -1.5902, -0.2751]],

        [[ 2.9091,  0.0371, -0.3771, -0.0621,  0.1385],
         [-0.6467,  1.8324,  1.4403,  1.0526,  1.4508],
         [ 1.1958, -1.2200,  1.6275,  1.0159, -0.6355],
         [ 0.9764,  0.3040, -0.9065,  1.4711,  2.9472],
         [ 1.6011,  0.1137, -0.4445,  0.7469,  0.0827]],

        [[ 0.0204,  0.6514, -1.3760, -0.6822,  1.0084],
         [ 0.7725,  0.2640, -0.1007,  1.2703, -0.1107],
         [ 0.1386, -1.8214, -1.9864,  1.3183,  0.5828],
         [ 0.0051, -1.8912,  0.7801, -1.8244, -0.8489],
         [-1.1395,  0.2634, -0.6095, -0.9424, -0.3488]]],
       names=('channels', 'rows', 'columns'))

In [43]:
batch_named

tensor([[[[-0.4327, -0.6534, -0.7317, -0.4231, -1.2641],
          [-2.5323, -0.6371, -0.0243,  0.7110, -1.0352],
          [ 2.6752, -0.2881, -0.2490,  2.1944, -0.7277],
          [-0.7770,  0.2503, -1.7195,  0.3949,  0.8570],
          [-0.8717,  1.3692,  0.5090, -1.6185,  0.7126]],

         [[-1.5003,  0.3588, -0.0232,  0.5664, -0.2286],
          [-1.5466, -0.5515, -0.7599,  0.3978, -0.9340],
          [-0.6201,  0.7426,  0.6493, -0.1922, -2.0722],
          [-0.0190, -0.5018,  0.9953,  0.6356,  0.4542],
          [-0.3126,  0.2169, -1.5904,  0.6665,  0.7936]],

         [[-0.8280,  0.3386, -0.0721,  1.4033,  0.3494],
          [-0.1538,  0.8704,  0.9939,  1.7908, -0.3326],
          [ 2.1390,  0.5973,  1.3204, -0.0993,  1.6589],
          [ 1.1751, -0.7440, -1.2444,  0.0924,  0.1416],
          [-0.6961,  2.0161, -2.1123,  1.4692,  0.2306]]],


        [[[-1.0430, -0.0069,  0.2654,  0.6894, -0.3216],
          [-0.2226,  0.9742,  0.3295,  0.3270, -0.2413],
          [ 0.5690,  1.

In [44]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

👆

对于已命名的张量, 也已通过 `align_as` 来让维度对齐, 让维度更低的向维度高的对齐. 之后就可以进行和之前相同的操作了.

In [46]:
img_gray_weighted_named = img_named * weights_aligned
batch_gray_weighted_named = batch_named * weights_aligned
img_gray_weighted_named.shape, batch_gray_weighted_named.shape

(torch.Size([3, 5, 5]), torch.Size([2, 3, 5, 5]))

#### P.S.

以下为错误示范

In [47]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


## 取消命名

In [50]:
img_gray_weighted_plain = img_gray_weighted_named.rename(None)
batch_gray_weighted_plain = batch_gray_weighted_named.rename(None)
img_gray_weighted_plain.shape, img_gray_weighted_plain.names, batch_gray_weighted_plain.shape, batch_gray_weighted_plain.names

(torch.Size([3, 5, 5]),
 (None, None, None),
 torch.Size([2, 3, 5, 5]),
 (None, None, None, None))